# Register to ARA

The aim is to register individual slices to the Allen Reference Atlas. Example solutions
for this type of registration are described here:
https://forum.image.sc/t/comparison-of-some-tools-for-brain-slice-to-atlas-registration/43260

We will start with ABBA. This require a stitched overview of one slice in a pyramidal format.

## Stitch example round

First step is to stitch on slice.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import iss_preprocess as iss
from tifffile import imread, imsave
import iss_preprocess as iss
from iss_preprocess.io.save import write_stack
import matplotlib.pyplot as plt
from flexiznam.config import PARAMETERS
import numpy as np
from pathlib import Path
from iss_preprocess.image import correction

project = "becalia_rabies_barseq"
mouse = "BRYC65.1d"
chamber = "chamber_13"
ref_acq = "genes_round_1_1"
# dark_frame = "becalia_iss1_microscope/calibration/20221209_dark_frame"

raw = Path(PARAMETERS["data_root"]["raw"])
data_path = Path(project) / mouse / chamber
processed = Path(PARAMETERS["data_root"]["processed"])
registration_folder = processed / data_path / "register_to_ara"
registration_folder.mkdir(exist_ok=True)


/camp/home/blota/.conda/envs/iss-preprocess/lib/python3.10/site-packages/trackpy/utils.py:22: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  is_pandas_since_023 = (LooseVersion(pd.__version__) >=
/camp/home/blota/.conda/envs/iss-preprocess/lib/python3.10/site-packages/trackpy/utils.py:23: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LooseVersion('0.23.0'))
/camp/home/blota/.conda/envs/iss-preprocess/lib/python3.10/site-packages/trackpy/utils.py:29: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  is_scipy_15 = LooseVersion("1.5.0") <= LooseVersion(scipy.__version__) < LooseVersion('1.6.0')
/camp/home/blota/.conda/envs/iss-preprocess/lib/python3.10/site-packages/trackpy/utils.py:29: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  is_scipy_15 = LooseVersion("1.5.0") <= LooseVersion(scipy.__version__)

### Make correction image

That should be done by the pipeline, but in case it's not done, here it is:

In [4]:
# Create single averages
REDO = False
average_dir = processed / data_path / "averages"
average_dir.mkdir(exist_ok=True)
if REDO:
    from iss_preprocess import pipeline
    pipeline.create_all_single_averages(data_path=data_path)

In [5]:
# Create grand averages
REDO = False
if REDO:
    from iss_preprocess import pipeline

    pipeline.create_grand_averages(
        data_path=data_path, prefix_todo=("genes_round", "barcode_round")
    )

In [6]:
# Do diagnostic plots
REDO = False
if REDO:
    from iss_preprocess.pipeline import diagnostic
    diagnostic.check_illumination_correction(
        data_path, grand_averages=("barcode_round", "genes_round")
    )

## Stitch tiles

We will take all rois in the `ref_acq` and stitch them. The save the generated image in
`registration_folder`.

In [7]:
reference_prefix = "genes_round_1_1" # including round
ops = np.load(processed / data_path / "ops.npy", allow_pickle=True).item()
suffix = ops['projection']

In [8]:
ops

{'camera_order': [1, 3, 4, 2],
 'genes_rounds': 7,
 'barcode_rounds': 10,
 'use_rois': [1, 2, 5, 6],
 'correction_tiles': [(1, 5, 8),
  (1, 5, 9),
  (1, 4, 8),
  (1, 4, 9),
  (2, 4, 9),
  (2, 3, 9),
  (2, 2, 9),
  (2, 2, 8)],
 'correction_quantile': 0.9999,
 'ref_tile': (1, 5, 8),
 'barcode_ref_tiles': [(1, 5, 8),
  (1, 5, 9),
  (1, 4, 8),
  (1, 4, 9),
  (2, 4, 9),
  (2, 3, 9),
  (2, 2, 9),
  (2, 2, 8)],
 'ref_ch': 0,
 'ref_round': 0,
 'filter_r': (2, 4),
 'detection_threshold': 0.15,
 'isolation_threshold': 0.05,
 'barcode_correct_channels': 'round1_only',
 'barcode_detection_threshold': 0.3,
 'barcode_isolation_threshold': 0.3,
 'barcode_detection_threshold_basecalling': 0.1,
 'hybridisation_correct_channels': True,
 'hyb_spot_detection_threshold': 0.5,
 'codebook': 'codebook_83gene_pool.csv',
 'projection': 'fstack',
 'hybridisation_projection': 'max',
 'spot_extraction_radius': 2,
 'spot_shape_radius': 7,
 'spot_shape_neighbor_filter_size': 9,
 'spot_shape_neighbor_threshold': 15,


In [7]:
# find the number of ROIs and the number of tiles per roi from file names
import re
from iss_preprocess.io.save import save_ome_tiff_pyramid

pattern = fr"{reference_prefix}_MMStack_(\d*)-Pos(\d\d\d)_(\d\d\d)_{suffix}.tif"
find_acq = re.compile(pattern)
data = dict()
for fname in (processed /  data_path / reference_prefix).iterdir():
    m = find_acq.match(fname.name)
    if m:
        nroi, tile_x, tile_y = np.array(m.groups(), dtype=int)
        if not nroi in data:
            data[nroi] = []
        data[nroi].append([tile_x, tile_y])
data = {k: np.vstack(v) for k, v in data.items()}
ntiles_per_roi = {k: np.max(v, axis=0) + 1 for k, v in data.items()}
print("Found %d ROIs." % len(ntiles_per_roi))

Found 8 ROIs.


In [8]:
metadata = iss.io.load.load_metadata(data_path)
acq_metadata = iss.io.load.load_single_acq_metdata(data_path, prefix='genes_round_1_1')
pixel_size = acq_metadata['FrameKey-0-0-0']['PixelSizeUm']
# pixel_size = 0.1862

### Find shifts

In [ ]:
shift_right, shift_down, tile_shape = iss.pipeline.register_adjacent_tiles(
    data_path, ref_coors=ops["ref_tile"], prefix=reference_prefix)

### Do the actual stitching

Of refrence channel only

In [ ]:
REDO = True
DO_ONE = False  # do only one and stop (useful to create variables to debug)
fig, axes = plt.subplots(int(np.ceil(len(ntiles_per_roi)/4)), 4)
for iroi, (roi_num, ntiles) in enumerate(ntiles_per_roi.items()):
    target = registration_folder / f"stitched_{reference_prefix}_roi_{roi_num}.ome.tif"
    if target.exists() and not REDO:
        print("%s already exists." % target.name, flush=True)
        continue
    print("Doing roi %d (%d by %d tiles)" % (roi_num, *ntiles), flush=True)
    print(" ... stitching", flush=True)
    
    stitched_stack = iss.pipeline.stitch_tiles(
        data_path,
        reference_prefix,
        shift_right,
        shift_down,
        suffix=ops["projection"],
        roi=roi_num,
        ich=ops['ref_ch'],
        correct_illumination=True,
    )
    # save output
    print(" ... saving reference channel %d" % ops['ref_ch'], flush=True)
    target = registration_folder / f"stitched_{reference_prefix}_roi_{roi_num}_ch{ops['ref_ch']}.ome.tif"
    smallest = save_ome_tiff_pyramid(
        target,
        stitched_stack,
        pixel_size=pixel_size,
        subresolutions=5,
        max_size=1,
    )
    axes.flatten()[iroi].imshow(smallest)
    axes.flatten()[iroi].set_title(f"Roi {roi_num}")
    if DO_ONE:
        break
    print("Done!")
for x in axes.flatten():
    x.set_xticks([])
    x.set_yticks([])
fig.tight_layout()


Doing roi 6 (19 by 10 tiles)
 ... stitching
 ... saving reference channel 0
... Clipping array
... Resize
... writing full res image
... writing pyramidal layer 1
... writing pyramidal layer 2
... writing pyramidal layer 3
... writing pyramidal layer 4
... writing pyramidal layer 5
Done!
Doing roi 7 (19 by 12 tiles)
 ... stitching
 ... saving reference channel 0
... Clipping array
... Resize
... writing full res image
... writing pyramidal layer 1
... writing pyramidal layer 2
... writing pyramidal layer 3
... writing pyramidal layer 4
... writing pyramidal layer 5
Done!
Doing roi 5 (18 by 12 tiles)
 ... stitching
 ... saving reference channel 0
... Clipping array
... Resize
... writing full res image
... writing pyramidal layer 1
... writing pyramidal layer 2
... writing pyramidal layer 3
... writing pyramidal layer 4
... writing pyramidal layer 5
Done!
Doing roi 4 (19 by 12 tiles)
 ... stitching


In [ ]:
projection = np.nanmax(out, axis=0)
plt.imshow(projection[100:10000:5, 5000:15000:5], vmax=200)


In [ ]:
projection = np.nanmin(out, axis=0)
plt.imshow(projection[::10, ::10])


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(smallest)


In [ ]:
import matplotlib.pyplot as plt

minp = np.nanmin(out, axis=0)


In [ ]:
print("Copy array")
fig = plt.figure(figsize=(15, 4))
image = np.array(minp)
plt.subplot(1, 3, 1)
plt.imshow(image[::20, ::20])
plt.colorbar()
print("Normalised")
p = np.nanquantile(image, 0.05)
q = np.nanquantile(image, 0.999)
image = (image - p) * 255 / (q - p)
plt.subplot(1, 3, 2)
plt.imshow(image[::20, ::20])
plt.colorbar()
print("Converting")
image = np.clip(image, 0, 255)
image = np.array(image, dtype="uint8")
plt.subplot(1, 3, 3)
plt.imshow(image[::20, ::20])
plt.colorbar()
